# Plotting in Julia

Based on a notebook by Chase Coleman and Spencer Lyon and on material from QuantEcon

18 December 2017

Overall message about plotting in Julia:
- Sort of a mixed bag
- Upside: many options, lots of flexibility
- Downside: many options, lots of flexibility

- Will introduce two options: 
    1. [PyPlot.jl](https://github.com/JuliaPy/PyPlot.jl): Julia interface to Python's [`matplotlib.pyplot`](http://matplotlib.org)
    2. [Plots.jl](https://juliaplots.github.io): API that leverages other packages to do plotting
- Alternatives
    - [Gadfly.jl](http://gadflyjl.org): Grammar of Graphics style plotting (similar to `ggplot2` from R)
    - [PlotlyJS.jl](http://spencerlyon.com/PlotlyJS.jl/): Interface to [plotly.js](https://plot.ly/javascript/)
    - [GLPlot.jl](https://github.com/SimonDanisch/GLPlot.jl): high performance GPU powered plotting
    - [GR.jl](https://github.com/jheinen/GR.jl): Julia interface to GR framework
    - ... others

## PyPlot.jl

- Julia interface to `matplotlib.pyplot`
- Very mature, robust, full-featured
- Main downside: sometimes feels a bit "clunky" or "unnatural" to use Python APIs from Julia

In [ ]:
# Pkg.add("PyPlot")

### Basic PyPlot

In [ ]:
using PyPlot

x = linspace(0, 2π, 30)
plot(x, sin.(x));  # semi-colon supresses matplotlib text

In [ ]:
### using object API
# One row, two columns of subplots
fig, ax = subplots(1, 2)

# 1d array of matplotlib axes
@show typeof(ax)

# access methods of ax object using `[:name]` instead of `.name`

# plot on both axes
ax[1][:plot](x, sin.(x), label="sin")
ax[2][:plot](x, cos.(x), label="cos")

# set axes properties
ax[1][:set_xlabel]("this is x")
ax[2][:set_title]("cos(x)")

# set figure property
fig[:suptitle]("Trig functions");

In [ ]:
# fix spacing of axes labels
fig[:tight_layout]()

# display fig again by having it as last item in notebook cell
# (notice improved spacing)
fig 

### More than you need

- There are many matplotlib tutorials and examples online
- Any that use `matplotlib.pyplot` work with PyPlot.jl -- you just need to change `.stuff` to `[:stuff]`
- The ultimate reference for what is possible is the matplotlib documentation:
    - [API documentation](http://matplotlib.org/2.0.2/api/pyplot_summary.html) (Documentation for each function)
    - [Examples](http://matplotlib.org/2.0.2/examples/index.html)
    - [Gallery](http://matplotlib.org/2.0.2/gallery.html)
- Below are some examples (taken from [quantecon plotting lecture](https://lectures.quantecon.org/jl/julia_plots.html#pyplot))

In [ ]:
using Distributions

u = Uniform()

fig, ax = subplots()
x = linspace(-4, 4, 150)
for i in 1:3
    # == Compute normal pdf from randomly generated mean and std == #
    m, s = rand(u) * 2 - 1, rand(u) + 1
    d = Normal(m, s)
    y = pdf(d, x)
    # == Plot current pdf == #
    ax[:plot](x, y, linewidth=2, alpha=0.6, label="draw $i")
end
ax[:legend]();

In [ ]:
u = Uniform()
num_rows, num_cols = 2, 3
fig, axes = subplots(num_rows, num_cols, figsize=(16,6))
subplot_num = 0

for i in 1:num_rows
    for j in 1:num_cols
        ax = axes[i, j]
        subplot_num += 1
        # == Generate a normal sample with random mean and std == #
        m, s = rand(u) * 2 - 1, rand(u) + 1
        d = Normal(m, s)
        x = rand(d, 100)
        # == Histogram the sample == #
        ax[:hist](x, alpha=0.6, bins=20)
        ax[:set_title]("histogram $subplot_num")
        ax[:set_xticks]([-4, 0, 4])
        ax[:set_yticks]([])
    end
end

In [ ]:
using QuantEcon: meshgrid

n = 50
x = linspace(-3, 3, n)
y = x

z = Array{Float64}(n, n)
f(x, y) = cos(x^2 + y^2) / (1 + x^2 + y^2)
for i in 1:n
    for j in 1:n
        z[j, i] = f(x[i], y[j])
    end
end

fig = figure(figsize=(8,6))
ax = fig[:gca](projection="3d")
ax[:set_zlim](-0.5, 1.0)
xgrid, ygrid = meshgrid(x, y)
ax[:plot_surface](
    xgrid, ygrid, z, rstride=2, cstride=2,
    cmap=ColorMap("viridis"), alpha=0.7, linewidth=0.25
);

In [ ]:
# save the previous figure to pdf and eps
fig[:savefig]("3dplot.pdf")
fig[:savefig]("3dplot.eps")

## Plots.jl

- A unified API for a wide variety of plotting packages (e.g. PlotlyJS, PyPlot, GR)
- Pros:
    - Only one set of syntax to learn
    - Easy to use 
    - Comprehensive set of backends
- Cons:
    - No guarantee that all features of a given backend will work

In [ ]:
# Pkg.add("Plots"); Pkg.add("GR")

### Plots.jl basics

The approach of Plots.jl is to

    1. provide a “frontend” plotting language
    2. render the plots by using one of several existing plotting libraries as “backends”

Plots.jl plotting commands are translated internally to commands understood by a selected plotting library. These underlying libraries, or backends, can be swapped easily. Each backend has a different look and a different set of capabilities.

For example, PyPlot, PlotlyJS, and GR ara available backends for Plots.jl. For this section, we will mostly focus on GR.

For complete documentation of Plots.jl check [here](http://docs.juliaplots.org/latest/).

**NOTE: restart notebook kernel here!!!**

Both Plots and PyPlot define functions with common names (e.g. `plot`, `scatter`, etc.) 

So that we can use Plots' without having to refer to them as `Plots.function_name` we will restart the notebook kernel (the Julia process backing the notebook)

To do this we go to the `Kernel` menu above and press `Restart`

In [ ]:
using Plots
gr()

In [ ]:
# a simple plot
x = linspace(0, 2π, 30)
plot(x, sin.(x), title="sin")

In [ ]:
# We can also arrange multiple plots in a grid
using Distributions
draws = Array{Vector}(4)
titles = Array{String}(1, 4)
for i = 1:4
    m, s = 2*(rand() - 0.5), rand() + 1
    d = Normal(m, s)
    draws[i] = rand(d, 100)
    t = "mu = $(round(m, 2)), sigma = $(round(s, 2))"
    titles[i] = t
end

Plots.histogram(draws,
            layout=4,
            title=titles,
            legend=:none,
            titlefont=Plots.font(9),
            bins=20)


### More than you need

Check the Plots.jl documentation for more information about how to manipulate and create plots to your liking. To give a taste of what other types of plots can be made using Plots.jl, see below:

In [ ]:
# a nice 3d plot
n = 100
ts = linspace(0,8π,n)
x = ts .* map(cos,ts)
y = (0.1ts) .* map(sin,ts)
z = 1:n
plot(x,y,z,zcolor=reverse(z),m=(10,0.8,:blues,stroke(0)),leg=false,cbar=true,w=5)
plot!(zeros(n),zeros(n),1:n,w=10)

In [ ]:
# a contour plot
x = 1:0.5:20
y = 1:0.5:10
f(x,y) = begin
        (3x + y ^ 2) * abs(sin(x) + cos(y))
    end
X = repmat(x',length(y),1)
Y = repmat(y,1,length(x))
Z = map(f,X,Y)
p1 = contour(x,y,f,fill=true)
p2 = contour(x,y,Z)
plot(p1,p2)